In [3]:
import torch
import numpy as np
import os
import torchvision
from torchvision import datasets, transforms

In [4]:
torch.cuda.is_available()

True

In [5]:
data_dir = '/content/drive/My Drive/flower_photos/'
train_dir = os.path.join(data_dir, 'train/')
test_dir = os.path.join(data_dir, 'test/')

classes = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [6]:
transform = torchvision.transforms.Compose([torchvision.transforms.RandomResizedCrop(224),
                                            torchvision.transforms.RandomHorizontalFlip(p = 0.5),
                                            torchvision.transforms.RandomVerticalFlip(p = 0.2),
                                            torchvision.transforms.RandomRotation((-45, 45)),
                                            torchvision.transforms.ToTensor()])

train_data = torchvision.datasets.ImageFolder(train_dir, transform = transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform = transform)

print (len(train_data))
print (len(test_data))

3139
540


In [7]:
batch_size = 20
num_workers = 0
indices = [i for i in range(0, len(train_data))]
np.random.shuffle(indices)
split = int(np.floor(0.8 * len(train_data)))
print(split)
train_idx, valid_idx = indices[:split], indices[split:]

2511


In [8]:
from torch.utils.data.sampler import SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers = num_workers, sampler = train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers = num_workers, sampler = valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, num_workers = num_workers)

In [9]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [11]:
class Net(nn.Module):
  def __init__(self):
    super (Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, 3, padding = 1)
    self.conv2 = nn.Conv2d(16, 64, 3, padding = 1)
    self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
    self.pool = nn.MaxPool2d(2, 2)
    self.fc1 = nn.Linear(128*28*28, 1000)
    self.fc2 = nn.Linear(1000, 100)
    self.fc3 = nn.Linear(100, 5)
    self.dropout = nn.Dropout(0.2)
  
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))

    x = x.view(-1, 128*28*28)
    x = self.dropout(x)
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = F.relu(self.fc2(x))
    x = self.dropout(x)
    x = self.fc3(x)
    return x

model = Net()
print(model)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=5, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)
n_epochs = 30
model.cuda()

for epoch in range(1, n_epochs + 1):
  train_loss = 0.0
  valid_loss = 0.0

  model.train()
  for data, target in train_loader:
    data, target = data.cuda(), target.cuda()
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  
  model.eval()
  for data, target in valid_loader:
    data, target = data.cuda(), target.cuda()
    output = model(data)
    loss = criterion(output, target)
    valid_loss += loss.item()*data.size(0)

  train_loss = train_loss/len(train_loader.sampler)
  valid_loss = valid_loss/len(valid_loader.sampler)

  print("Epoch {}, Training Loss {:.6f}, Validation Loss {:.6f}".format(epoch, train_loss, valid_loss))

Epoch 1, Training Loss 1.599550, Validation Loss 1.606935
Epoch 2, Training Loss 1.595628, Validation Loss 1.603059
Epoch 3, Training Loss 1.587179, Validation Loss 1.583716
Epoch 4, Training Loss 1.563900, Validation Loss 1.571642
Epoch 5, Training Loss 1.513712, Validation Loss 1.532923
Epoch 6, Training Loss 1.435778, Validation Loss 1.352560
Epoch 7, Training Loss 1.331724, Validation Loss 1.354478
Epoch 8, Training Loss 1.263634, Validation Loss 1.286020
Epoch 9, Training Loss 1.235408, Validation Loss 1.246190
Epoch 10, Training Loss 1.227602, Validation Loss 1.205172
Epoch 11, Training Loss 1.205229, Validation Loss 1.198198
Epoch 12, Training Loss 1.199090, Validation Loss 1.197395
Epoch 13, Training Loss 1.181949, Validation Loss 1.148935
Epoch 14, Training Loss 1.167832, Validation Loss 1.138760
Epoch 15, Training Loss 1.144218, Validation Loss 1.169638
Epoch 16, Training Loss 1.137426, Validation Loss 1.213935
Epoch 17, Training Loss 1.123340, Validation Loss 1.063961
Epoch 

In [16]:
test_loss = 0.0
class_correct = [0 for i in range(0, 5)]
class_total = [0 for i in range(0, 5)]

In [18]:
model.eval()
for data, target in test_loader:
  data, target = data.cuda(), target.cuda()
  output = model(data)
  loss = criterion(output, target)
  test_loss = test_loss + loss.item()*data.size(0)
  _, pred = torch.max(output, 1)
  correct_tensor = pred.eq(target.data.view_as(pred))
  correct = np.squeeze(correct_tensor.cpu().numpy())

  for i in range(0, batch_size):
    label = target.data[i]
    class_correct[label] += correct[i].item()
    class_total[label] += 1

In [19]:
test_loss = test_loss/len(test_loader.dataset)
print(test_loss)

0.9941896023573699


In [20]:
for i in range(5):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of daisy: 67% (62/92)
Test Accuracy of dandelion: 63% (84/132)
Test Accuracy of roses: 13% (12/91)
Test Accuracy of sunflowers: 81% (82/101)
Test Accuracy of tulips: 62% (78/124)

Test Accuracy (Overall): 58% (318/540)
